In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.preprocessing import image
from IPython.display import Image
import matplotlib.pyplot as plt
import os

In [ ]:
DATADIR = 'train0,1'

CATEGORIES = ['n', 'p']

training_data = []
WIDTH = 50
HIGHT = 150
img_size = HIGHT * WIDTH

In [ ]:
def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)  # path to each class dir
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = image.load_img(os.path.join(path, img), grayscale=True, target_size=(HIGHT, WIDTH))
                training_data.append([img_array, class_num])
            except:
                pass
            

create_training_data()

In [ ]:
import random

random.shuffle(training_data)
random.shuffle(testing_data)
train_images = []
train_labels = []
for i in range(len(training_data)):
    train_images.append(image.img_to_array(training_data[i][0]))
    train_labels.append(training_data[i][1])
random.shuffle(testing_data)
test_images = []
test_labels = []
for i in range(len(testing_data)):
    test_images.append(image.img_to_array(testing_data[i][0]))
    test_labels.append(testing_data[i][1])

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
hight = 150
width = 50


train_images = np.array(train_images)
test_images = np.array(test_images)

In [ ]:
if K.image_data_format() == 'channels_first':
    train_images = train_images.reshape(train_images.shape[0], 1, hight, width)
    test_images = test_images.reshape(test_images.shape[0], 1, hight, width)
    input_shape = (1, hight, width)
else:
    train_images = train_images.reshape(train_images.shape[0], hight, width, 1)
    test_images = test_images.reshape(test_images.shape[0], hight, width, 1)
    input_shape = (hight, width, 1)

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
# 64 3x3 kernels 
model.add(Conv2D(64, (3, 3), activation='relu'))
# Reduce by taking the max of each 2x2 block
model.add(MaxPooling2D(pool_size=(2, 2)))
# Dropout to avoid overfitting
model.add(Dropout(0.25))
# 64 3x3 kernels 
model.add(Conv2D(64, (3, 3), activation='relu'))
# Flatten the results to one dimension for passing into out final layer
model.add(Flatten())
# A hidden layer to learn with
model.add(Dense(128, activation='relu'))
# A hidden layer to learn with
model.add(Dense(128, activation='relu'))
# Another dropout
model.add(Dropout(0.5))
# Final categorization from 0-1 with softmax
model.add(Dense(2))
model.add(keras.layers.Softmax(axis=-1))
model.compile(loss=keras.losses.categorical_crossentropy,

              optimizer=keras.optimizers.Adam(lr=0.00001),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit(train_images , train_new_labels,
                 batch_size=16,
                 epochs=10,
                 verbose=2,
                 validation_data=(test_images,test_new_labels))

In [ ]:
score=model.evaluate(test_images,test_new_labels , verbose=0)
print('Test loss :',score[0])
print('Test accuracy :',score[1]) 
model.save('PhoneDetection-CNN_3_April.h5')